<a href="https://colab.research.google.com/github/map72ru/matalg/blob/main/%D0%94%D0%974.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
1. В коде из методички реализуйте один или несколько из критериев останова (количество листьев, количество используемых признаков, глубина дерева и т.д.).

Добавлен класс Conditions и реализованы остановки по глубине дерева, количеству объектов в листе (как параметр), количеству листов, в так же по уровню ошибки

In [ ]:
import math

from sklearn import datasets
import numpy as np


# Реализуем класс узла

class Node:

    def __init__(self, index, t, true_branch, false_branch):
        self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
        self.t = t  # значение порога
        self.true_branch = true_branch  # поддерево, удовлетворяющее условию в узле
        self.false_branch = false_branch  # поддерево, не удовлетворяющее условию в узле


# И класс терминального узла (листа)

class Leaf:

    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.prediction = self.predict()

    def predict(self):
        # подсчет количества объектов разных классов
        classes = {}  # сформируем словарь "класс: количество объектов"
        for label in self.labels:
            if label not in classes:
                classes[label] = 0
            classes[label] += 1
        #  найдем класс, количество объектов которого будет максимальным в этом листе и вернем его
        prediction = max(classes, key=classes.get)
        return prediction

class Conditions:
    max_levels = -1
    max_leafs = -1
    min_quality = -1
    min_objects = 5

    __cur_leafs = 0
    __cur_levels = 0

    def inc_leaf(self):
        self.__cur_leafs += 1

    def apply(self):
        if self.max_levels > -1 and self.__cur_levels >= self.max_levels:
            return True

        if self.max_leafs > -1 and self.__cur_leafs >= self.max_leafs:
            return True

        self.__cur_levels += 1
        return False

    def check_quality(self, current_quality):
        return self.min_quality > -1 and current_quality <= self.min_quality

# Расчет критерия Джини

def gini(labels):
    #  подсчет количества объектов разных классов
    classes = {}
    for label in labels:
        if label not in classes:
            classes[label] = 0
        classes[label] += 1

    #  расчет критерия
    impurity = 1
    for label in classes:
        p = classes[label] / len(labels)
        impurity -= p ** 2

    return impurity

def entropy(labels):
    classes = {}
    for label in labels:
        if label not in classes:
            classes[label] = 0
        classes[label] += 1

    #  расчет критерия
    entropy = 0
    for label in classes:
        p = classes[label] / len(labels)
        entropy += p * math.log(p, 2)

    return entropy

# Расчет качества

def quality(left_labels, right_labels, current_crit, criteria):
    # доля выбоки, ушедшая в левое поддерево
    p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0])

    return current_crit - p * criteria(left_labels) - (1 - p) * criteria(right_labels)

# Разбиение датасета в узле

def split(data, labels, index, t):
    left = np.where(data[:, index] <= t)
    right = np.where(data[:, index] > t)

    true_data = data[left]
    false_data = data[right]
    true_labels = labels[left]
    false_labels = labels[right]

    return true_data, false_data, true_labels, false_labels

# Нахождение наилучшего разбиения

def find_best_split(data, labels, conditions, criteria):
    #  обозначим минимальное количество объектов в узле
    if conditions is not None:
        min_leaf = conditions.min_objects
    else:
        min_leaf = 5

    current_crit = criteria(labels)

    best_quality = 0
    best_t = None
    best_index = None

    n_features = data.shape[1]

    for index in range(n_features):
        # будем проверять только уникальные значения признака, исключая повторения
        t_values = np.unique([row[index] for row in data])

        for t in t_values:
            true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
            #  пропускаем разбиения, в которых в узле остается менее 5 объектов
            if len(true_data) < min_leaf or len(false_data) < min_leaf:
                continue

            current_quality = quality(true_labels, false_labels, current_crit, criteria)

            #  выбираем порог, на котором получается максимальный прирост качества
            if current_quality > best_quality:
                best_quality, best_t, best_index = current_quality, t, index

    return best_quality, best_t, best_index

# Построение дерева с помощью рекурсивной функции

def build_tree(data, labels, conditions = None, criteria = gini):

    # Ограничения по дереву
    if conditions is not None:
        if conditions.apply():
            return Leaf(data, labels)

    quality, t, index = find_best_split(data, labels, conditions, criteria)

    if conditions is not None:
        if conditions.check_quality(quality):
            conditions.inc_leaf()
            return Leaf(data, labels)

    #  Базовый случай - прекращаем рекурсию, когда нет прироста в качества
    if quality == 0:
        if conditions is not None:
            conditions.inc_leaf()
        return Leaf(data, labels)

    true_data, false_data, true_labels, false_labels = split(data, labels, index, t)

    # Рекурсивно строим два поддерева
    true_branch = build_tree(true_data, true_labels, conditions)
    false_branch = build_tree(false_data, false_labels, conditions)

    # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
    return Node(index, t, true_branch, false_branch)

def classify_object(obj, node):

    #  Останавливаем рекурсию, если достигли листа
    if isinstance(node, Leaf):
        answer = node.prediction
        return answer

    if obj[node.index] <= node.t:
        return classify_object(obj, node.true_branch)
    else:
        return classify_object(obj, node.false_branch)


def predict(data, tree):
    classes = []
    for obj in data:
        prediction = classify_object(obj, tree)
        classes.append(prediction)
    return classes



In [ ]:
# Разобьем выборку на обучающую и тестовую

from sklearn import model_selection

# сгенерируем данные
classification_data, classification_labels = datasets.make_classification(n_samples=1000, n_features = 3, n_informative = 3,
                                                      n_classes = 3, n_redundant=0,
                                                      n_clusters_per_class=1, random_state=5)

train_data, test_data, train_labels, test_labels = model_selection.train_test_split(classification_data,
                                                                                     classification_labels,
                                                                                     test_size = 0.3,
                                                                                     random_state = 1)



Просто проверим, что все работает

In [ ]:
# Построим дерево по обучающей выборке
my_tree = build_tree(train_data, train_labels)

# Напечатаем ход нашего дерева
def print_tree(node, spacing=""):
    # Если лист, то выводим его прогноз
    if isinstance(node, Leaf):
        print(spacing + "Прогноз:", node.prediction)
        return

    # Выведем значение индекса и порога на этом узле
    print(spacing + 'Индекс', str(node.index))
    print(spacing + 'Порог', str(node.t))

    # Рекурсионный вызов функции на положительном поддереве
    print(spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Рекурсионный вызов функции на положительном поддереве
    print(spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")


print_tree(my_tree)

# Получим ответы для обучающей выборки
train_answers = predict(train_data, my_tree)

# И получим ответы для тестовой выборки
answers = predict(test_data, my_tree)

# Введем функцию подсчета точности как доли правильных ответов
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

# Точность на обучающей выборке
train_accuracy = accuracy_metric(train_labels, train_answers)
print('train_accuracy',train_accuracy)

# Точность на тестовой выборке
test_accuracy = accuracy_metric(test_labels, answers)
print('test_accuracy', test_accuracy)


Индекс 1
Порог 0.291136309339993
--> True:
  Индекс 0
  Порог -0.7094546141261373
  --> True:
    Индекс 2
    Порог -4.03544097413468
    --> True:
      Прогноз: 1
    --> False:
      Прогноз: 1
  --> False:
    Индекс 1
    Порог -0.16234689302965566
    --> True:
      Индекс 0
      Порог -0.21733916492530092
      --> True:
        Прогноз: 2
      --> False:
        Индекс 2
        Порог -3.163219683887811
        --> True:
          Прогноз: 2
        --> False:
          Индекс 0
          Порог 3.059135223077212
          --> True:
            Прогноз: 2
          --> False:
            Прогноз: 2
    --> False:
      Индекс 0
      Порог 2.0029661862505383
      --> True:
        Индекс 0
        Порог -0.056964246499989324
        --> True:
          Индекс 2
          Порог -1.7714876459826936
          --> True:
            Прогноз: 2
          --> False:
            Прогноз: 1
        --> False:
          Индекс 1
          Порог 0.07344292828091659
          --> True:

Введем ограничения по глубине дерева

In [ ]:

#
# Построим дерево с ограничениями

conditions = Conditions()
conditions.max_levels = 3
# Построим дерево по обучающей выборке
my_tree = build_tree(train_data, train_labels, conditions)

print_tree(my_tree)

Индекс 1
Порог 0.291136309339993
--> True:
  Индекс 0
  Порог -0.7094546141261373
  --> True:
    Индекс 2
    Порог -4.03544097413468
    --> True:
      Прогноз: 1
    --> False:
      Прогноз: 1
  --> False:
    Прогноз: 2
--> False:
  Прогноз: 0


2. Для задачи классификации обучить дерево решений с использованием критериев разбиения Джини и Энтропия. Сравнить качество классификации, сделать выводы.

Посчитаем с критерием по энтропии Шеннона и сравним с первым результатом

In [ ]:
# Построим дерево по обучающей выборке
my_tree = build_tree(train_data, train_labels, criteria=entropy)

# Получим ответы для обучающей выборки
entropy_train_answers = predict(train_data, my_tree)

# И получим ответы для тестовой выборки
answers = predict(test_data, my_tree)

# Точность на обучающей выборке
entropy_train_accuracy = accuracy_metric(train_labels, entropy_train_answers)
print('entropy_train_accuracy',entropy_train_accuracy)

# Точность на тестовой выборке
entropy_test_accuracy = accuracy_metric(test_labels, answers)
print('entropy_test_accuracy', entropy_test_accuracy)


entropy_train_accuracy 33.85714285714286
entropy_test_accuracy 31.666666666666664


На данной выборке (1000 измерений, 3 признака) критерий энтропии Шеннона дает результат значительно хуже, чем критерий Джини. Пробовал увеличивать/уменьшать размер выборки и кол-во признаков, отношение значений двух критериев кардинально не менялось. Скорее всего причина в самой сути данных.

3 [опция]. Реализуйте дерево для задачи регрессии. Возьмите за основу дерево, реализованное в методичке, заменив механизм предсказания в листе на взятие среднего значения по выборке, и критерий Джини на дисперсию значений.

In [ ]:
import math

from sklearn import datasets
import numpy as np


# Реализуем класс узла

class Node:

    def __init__(self, index, t, true_branch, false_branch):
        self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
        self.t = t  # значение порога
        self.true_branch = true_branch  # поддерево, удовлетворяющее условию в узле
        self.false_branch = false_branch  # поддерево, не удовлетворяющее условию в узле


# И класс терминального узла (листа)

class Leaf:

    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.prediction = self.predict()

    def predict(self):
        return np.mean(self.labels)

class Conditions:
    max_levels = -1
    max_leafs = -1
    min_quality = -1
    min_objects = 5

    __cur_leafs = 0
    __cur_levels = 0

    def inc_leaf(self):
        self.__cur_leafs += 1

    def apply(self):
        if self.max_levels > -1 and self.__cur_levels >= self.max_levels:
            return True

        if self.max_leafs > -1 and self.__cur_leafs >= self.max_leafs:
            return True

        self.__cur_levels += 1
        return False

    def check_quality(self, current_quality):
        return self.min_quality > -1 and current_quality <= self.min_quality

# Расчет критерия Джини

def gini(labels):
    #  подсчет количества объектов разных классов
    classes = {}
    for label in labels:
        if label not in classes:
            classes[label] = 0
        classes[label] += 1

    #  расчет критерия
    impurity = 1
    for label in classes:
        p = classes[label] / len(labels)
        impurity -= p ** 2

    return impurity

def variance(labels):
    mean = np.mean(labels)
    return np.sum((labels-mean)**2)/len(labels)

def entropy(labels):
    classes = {}
    for label in labels:
        if label not in classes:
            classes[label] = 0
        classes[label] += 1

    #  расчет критерия
    entropy = 0
    for label in classes:
        p = classes[label] / len(labels)
        entropy += p * math.log(p, 2)

    return entropy

# Расчет качества

def quality(left_labels, right_labels, current_crit, criteria):
    # доля выбоки, ушедшая в левое поддерево
    p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0])

    return current_crit - p * criteria(left_labels) - (1 - p) * criteria(right_labels)

# Разбиение датасета в узле

def split(data, labels, index, t):
    left = np.where(data[:, index] <= t)
    right = np.where(data[:, index] > t)

    true_data = data[left]
    false_data = data[right]
    true_labels = labels[left]
    false_labels = labels[right]

    return true_data, false_data, true_labels, false_labels

# Нахождение наилучшего разбиения

def find_best_split(data, labels, conditions, criteria):
    #  обозначим минимальное количество объектов в узле
    if conditions is not None:
        min_leaf = conditions.min_objects
    else:
        min_leaf = 5

    current_crit = criteria(labels)

    best_quality = 0
    best_t = None
    best_index = None

    n_features = data.shape[1]

    for index in range(n_features):
        # будем проверять только уникальные значения признака, исключая повторения
        t_values = np.unique([row[index] for row in data])

        for t in t_values:
            true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
            #  пропускаем разбиения, в которых в узле остается менее 5 объектов
            if len(true_data) < min_leaf or len(false_data) < min_leaf:
                continue

            current_quality = quality(true_labels, false_labels, current_crit, criteria)

            #  выбираем порог, на котором получается максимальный прирост качества
            if current_quality > best_quality:
                best_quality, best_t, best_index = current_quality, t, index

    return best_quality, best_t, best_index

# Построение дерева с помощью рекурсивной функции

def build_tree(data, labels, conditions = None, criteria = gini):

    # Ограничения по дереву
    if conditions is not None:
        if conditions.apply():
            return Leaf(data, labels)

    quality, t, index = find_best_split(data, labels, conditions, criteria)

    if conditions is not None:
        if conditions.check_quality(quality):
            conditions.inc_leaf()
            return Leaf(data, labels)

    #  Базовый случай - прекращаем рекурсию, когда нет прироста в качества
    if quality == 0:
        if conditions is not None:
            conditions.inc_leaf()
        return Leaf(data, labels)

    true_data, false_data, true_labels, false_labels = split(data, labels, index, t)

    # Рекурсивно строим два поддерева
    true_branch = build_tree(true_data, true_labels, conditions)
    false_branch = build_tree(false_data, false_labels, conditions)

    # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
    return Node(index, t, true_branch, false_branch)

def classify_object(obj, node):

    #  Останавливаем рекурсию, если достигли листа
    if isinstance(node, Leaf):
        answer = node.prediction
        return answer

    if obj[node.index] <= node.t:
        return classify_object(obj, node.true_branch)
    else:
        return classify_object(obj, node.false_branch)


def predict(data, tree):
    classes = []
    for obj in data:
        prediction = classify_object(obj, tree)
        classes.append(prediction)
    return classes

# Разобьем выборку на обучающую и тестовую

In [ ]:
from sklearn import model_selection

# сгенерируем данные
classification_data, classification_labels = datasets.make_regression(n_samples=1000, n_features = 3, n_informative = 3,
                                                                      noise=0.1)

train_data, test_data, train_labels, test_labels = model_selection.train_test_split(classification_data,
                                                                                     classification_labels,
                                                                                     test_size = 0.3,
                                                                                     random_state = 1)

In [ ]:
tree = build_tree(train_data, train_labels, criteria=variance)


In [ ]:
# В качестве метрики возьмем коэффициент детерминации
def determination(y, y_predict):
  return 1-np.sum((y-y_predict)**2)/np.sum((y-np.mean(y))**2)


# Получим ответы для обучающей выборки
var_train_answers = predict(train_data, tree)

# И получим ответы для тестовой выборки
var_answers = predict(test_data, tree)

# Точность на обучающей выборке
var_train_accuracy = determination(train_labels, var_train_answers)
print('var_train_accuracy',var_train_accuracy)

# Точность на тестовой выборке
var_test_accuracy = determination(test_labels, var_answers)
print('var_test_accuracy', var_test_accuracy)

var_train_accuracy 0.7279230632634396
var_test_accuracy 0.6413070032850283
